In [22]:
import requests
import json

# Define the API endpoint and payload
endpoint = "http://localhost:11434/api/generate"
payload = {
    "prompt": "What is the capital of Vietnam?",
    "model": "llama3:8b",
    "temperature": 0.6,
    "max_new_tokens": 256
}

# Convert the payload to JSON
payload_json = json.dumps(payload)

# Set the headers
headers = {
    'Content-Type': 'application/json'
}

# Send the POST request
response = requests.post(endpoint, headers=headers, data=payload_json)

# Check if the request was successful
if response.status_code == 200:
    try:
        # Split the response into lines and parse each line as JSON
        response_lines = response.text.strip().split("\n")
        full_response = ""
        for line in response_lines:
            response_json = json.loads(line)  # Parse each line as JSON
            full_response += response_json.get("response", "")  # Concatenate the "response" field
        print("Full Response:", full_response)
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
else:
    print("Failed to retrieve response:", response.status_code)

Full Response: The capital of Vietnam is Hanoi.


In [8]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([('VisitAsia', 'Tuberculosis'), 
                         ('Smoking', 'LungCancer'), 
                         ('Smoking', 'Bronchitis'), 
                         ('Tuberculosis', 'Either'), 
                         ('LungCancer', 'Either'), 
                         ('Either', 'ChestXray'), 
                         ('Either', 'Dyspnea'), 
                         ('Bronchitis', 'Dyspnea')])

# Add CPDs
cpd_visit_asia = TabularCPD(variable='VisitAsia', variable_card=2, values=[[0.99], [0.01]])
cpd_smoking = TabularCPD(variable='Smoking', variable_card=2, values=[[0.5], [0.5]])
cpd_tuberculosis = TabularCPD(variable='Tuberculosis', variable_card=2,
                              values=[[0.99, 0.95], [0.01, 0.05]],
                              evidence=['VisitAsia'], evidence_card=[2])
cpd_lung_cancer = TabularCPD(variable='LungCancer', variable_card=2,
                             values=[[0.99, 0.9], [0.01, 0.1]],
                             evidence=['Smoking'], evidence_card=[2])
cpd_bronchitis = TabularCPD(variable='Bronchitis', variable_card=2,
                            values=[[0.7, 0.4], [0.3, 0.6]],
                            evidence=['Smoking'], evidence_card=[2])
cpd_either = TabularCPD(variable='Either', variable_card=2,
                        values=[[1, 0, 0, 0], [0, 1, 1, 1]],
                        evidence=['Tuberculosis', 'LungCancer'],
                        evidence_card=[2, 2])
cpd_chest_xray = TabularCPD(variable='ChestXray', variable_card=2,
                            values=[[0.95, 0.02], [0.05, 0.98]],
                            evidence=['Either'], evidence_card=[2])
cpd_dyspnea = TabularCPD(variable='Dyspnea', variable_card=2,
                         values=[[0.9, 0.7, 0.8, 0.1], [0.1, 0.3, 0.2, 0.9]],
                         evidence=['Either', 'Bronchitis'], evidence_card=[2, 2])

# Add CPDs to the model
model.add_cpds(cpd_visit_asia, cpd_smoking, cpd_tuberculosis, cpd_lung_cancer, 
               cpd_bronchitis, cpd_either, cpd_chest_xray, cpd_dyspnea)

# Validate the model
assert model.check_model()

def print_cpds(model):
    for cpd in model.get_cpds():
        print("CPD of {variable}:".format(variable=cpd.variable))
        print(cpd)

# print_cpds(model)

# # Perform inference
inference = VariableElimination(model)

# # Query 1: Probability of having Tuberculosis
# query_result_1 = inference.query(variables=['Tuberculosis'])
# print("Probability of Tuberculosis:")
# print(query_result_1)

# # Query 2: Probability of Dyspnea given Smoking = 1
# query_result_2 = inference.query(variables=['Dyspnea'], evidence={'Smoking': 1})
# print("\nProbability of Dyspnea given Smoking = 1:")
# print(query_result_2)

# # Query 3: Probability of ChestXray being positive given Either = 1
# query_result_3 = inference.query(variables=['ChestXray'], evidence={'Either': 1})
# print("\nProbability of ChestXray being positive given Either = 1:")
# print(query_result_3)

# # Query 4: Probability of Bronchitis given Smoking = 1
query_result_4 = inference.query(variables=['Bronchitis'], evidence={'Smoking': 1})
print("\nProbability of Bronchitis given Smoking = 1:")
print(query_result_4)


Probability of Bronchitis given Smoking = 1:
+---------------+-------------------+
| Bronchitis    |   phi(Bronchitis) |
+===============+===================+
| Bronchitis(0) |            0.4000 |
+---------------+-------------------+
| Bronchitis(1) |            0.6000 |
+---------------+-------------------+
